In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
#Advanced Fork-Join Example 12#Complex fork-join with 2 parallel branches
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.FCFS)
queue2 = Queue(model,'Queue2',SchedStrategy.FCFS)
queue3 = Queue(model,'Queue3',SchedStrategy.FCFS)
queue4 = Queue(model,'Queue4',SchedStrategy.FCFS)
queue5 = Queue(model,'Queue5',SchedStrategy.FCFS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(2.0))
queue3.set_service(jobclass1, Exp(1.0))
queue4.set_service(jobclass1, Exp(3.0))
queue5.set_service(jobclass1, Exp(0.8))
delay.set_service(jobclass1, Exp(0.5))

In [5]:
# Set routing matrix with complex serial routing
P = model.init_routing_matrix()

P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
# Complex serial routing:
P.set(jobclass1, jobclass1, queue1, queue4, 1.0)  # queue1 -> queue4
P.set(jobclass1, jobclass1, queue4, queue5, 1.0)  # queue4 -> queue5  
P.set(jobclass1, jobclass1, queue5, join, 1.0)    # queue5 -> join
P.set(jobclass1, jobclass1, queue2, queue3, 1.0)  # queue2 -> queue3
P.set(jobclass1, jobclass1, queue3, join, 1.0)    # queue3 -> join
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [JMT(model, seed=23000), MVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.avg_table()

SOLVER 1: JMT


Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Delay1   class1 1.5069 1.5069 1.9816  1.9816 0.7620 0.7620
 Queue1   class1 2.6112 0.7700 3.3819  3.3819 0.7619 0.7619
 Queue2   class1 0.5889 0.3630 0.7839  0.7839 0.7621 0.7621
 Queue3   class1 2.5048 0.7672 3.3668  3.3668 0.7617 0.7617
 Queue4   class1 0.3540 0.2490 0.4518  0.4518 0.7619 0.7619
 Queue5   class1 5.5023 0.9510 7.0491  7.0491 0.7713 0.7713
   Join   class1 5.5342 0.9782 3.5883  3.5883 0.7619 0.7619
SOLVER 2: MVA
Station JobClass    QLen   Util   RespT  ResidT   ArvR   Tput
 Delay1   class1  1.4136 1.4136  2.0000  2.0000 0.7068 0.7068
 Queue1   class1  2.4106 0.7068  3.4106  3.4106 0.7068 0.7068
 Queue2   class1  0.5465 0.3534  0.7733  0.7733 0.7068 0.7068
 Queue3   class1  2.4106 0.7068  3.4106  3.4106 0.7068 0.7068
 Queue4   class1  0.3082 0.2356  0.4361  0.4361 0.7068 0.7068
 Queue5   class1  7.5832 0.8835 10.7290 10.7290 0.7068 0.7068
   Join   class1 29.6730 0.0000  8.3965  8.3965 0.7068 0.7068
